# Métricas de Avaliação para Modelos de Linguagem

Modelos de linguagem podem ser aplicados em diversas tarefas de processamento de linguagem natural. Para avaliar o desempenho de tais modelos e comparar suas performances nessas tarefas, diversas métricas são utilizadas. As métricas permitem quantificar objetivamente o desempenho na tarefa e fornece uma base comum de comparação. Além disso, análises mais detalhadas dos casos de erro mais extremos podem fornecer informações úteis para implementação de melhorias nas arquiteturas.

As métricas de avaliação necessitam também de um conjunto de dados para teste. O modelo em si, sem o fornecimento de dados de entrada e uma saída esperada, não fornece informação que permite julgar o seu desempenho, dessa forma conjuntos de dados são criados para a avaliação das tarefas. Como se fosse uma prova que aplicada para várias pessoas e corrigida pelo mesmo grupo de avaliadores, garante que a performance de todos seja comparável.

# Dependências

In [1]:
!pip install datasets transformers evaluate sentencepiece sacremoses sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [2]:
import transformers
transformers.logging.set_verbosity_error()

# Evaluator

In [3]:
import evaluate
from datasets import load_dataset
from transformers import pipeline
from datasets import load_dataset
import random

## Perplexidade

As métricas intrínsicas avaliam o desempenho de modelos de linguagem em sua tarefa básica de treinamento, ou seja, capturar a distribuição estatística dos dados. Em modelos de linguagem auto-regressivo (GPT, LaMDA, PaLM) o modelamento estatístico busca definir a probabilidade do próximo token da sequência, atribuindo alta probabilidade aos tokens mais prováveis.

A medida de perplexidade exprime o nível de incerteza que o modelo possui ao definir a distribuição de probabilidade do próximo token dado uma sequência anterior. Assim, quanto maior a perplexidade mais incerto o modelo é sobre sua predição, distribuindo mais a probabilidade entre diversos tokens, ou seja, ao invés de ter somente um bom candidato para continuação da sequência são preditos múltiplos candidatos. É importante notar nessa definição que a perplexidade é somente uma medida de confiança do modelo quanto sua predição, mas **não** da sua acurácia. Formalmente a perplexidade é dada por:

$$
e^{- \frac{1}{N} \sum_{1}^{N} ln\, p_{\theta}(x_i|x_{i-1},...,x_{0})} = \prod_{1}^{N} p_{\theta}(x_i|x_{i-1},...,x_{0})^{-1/N}
$$

Onde $x_i$ é o token de maior probabilidade para sequência, $p_\theta$ é a probabilidade de um token dado uma sequência anterior e $N$ é o total de tokens na sequência.

Abaixo iremos testar três modelos com a mesma arquitetura, mas número de parâmetros diferentes. Para isso, utilizamos o dataset [IMDB]() que contem reviews de filmes e o módulo `evaluate` do Hugging Face para computar automaticamente a métrica.



In [5]:
import random
import evaluate
from datasets import load_dataset

# Load and shuffle the dataset
dataset = load_dataset("imdb", split="test").shuffle()

# Select a random subset of 500 examples
n = len(dataset)
subset = random.sample(range(n), k=500)
dataset_texts = dataset.select(subset)["text"]

# Load the perplexity metric
perplexity = evaluate.load("perplexity", module_type="metric")

# Compute perplexity for each model
results_70m = perplexity.compute(model_id='EleutherAI/pythia-70m-deduped',
                                 predictions=dataset_texts,
                                 batch_size=8)
results_160m = perplexity.compute(model_id='EleutherAI/pythia-160m-deduped',
                                  predictions=dataset_texts,
                                  batch_size=8)
results_410m = perplexity.compute(model_id='EleutherAI/pythia-410m-deduped',
                                  predictions=dataset_texts,
                                  batch_size=2)


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Abaixo podemos ver os resultados de perplexidade para cada modelo.

In [7]:
print('Pythia 70M: ' , results_70m['mean_perplexity'])
print('Pythia 160M: ' , results_160m['mean_perplexity'])
print('Pythia 410M: ' , results_410m['mean_perplexity'])

Pythia 70M:  72.66938973999024
Pythia 160M:  68.24382973480225
Pythia 410M:  32.95471157073975


## SQuAD V2

O Stanford Question Answering Dataset (SQuAD) é um conjunto de dados de triplas contendo um contexto, uma pergunta e uma resposta voltado para a avaliação da capacidade de compreensão de linguagem. Em cada tripla, a resposta é um trecho do texto de contexto que responde a pergunta. A tarefa do modelo é, a partir da sequência de entrada composta pelo contexto e pergunta, gerar a posição inicial e final do trecho do contexto que responde a pergunta. Adicionalmente, no SQuAD v2 existem perguntas que não podem ser respondidas a partir do contexto fornecido e o modelo deve ser capaz de indicar que não há resposta para pergunta.

O desempenho do modelo no dataset é medido com a métrica F1, a qual indica de maneira geral a qualidade do modelo em acertar o trecho da resposta. Pode-se também analisar a métrica F1 separadamente para os dados que possuem resposta e os que não podem ser respondidos.

In [8]:
qna_model_tiny = 'deepset/tinyroberta-squad2'
qna_model_base = 'deepset/roberta-base-squad2'
qna_model_large = 'deepset/roberta-large-squad2'

squad_v2_dataset = load_dataset('squad_v2', split='validation')

# Seleciona um subconjunto aleatório do dataset
n = len(squad_v2_dataset)
subset = random.choices(range(n), k=1000)
squad_v2_dataset = squad_v2_dataset.select(subset)

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [9]:
qna_eval = evaluate.evaluator('question-answering')

results_tiny = qna_eval.compute(model_or_pipeline=qna_model_tiny,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

results_base = qna_eval.compute(model_or_pipeline=qna_model_base,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

results_large = qna_eval.compute(model_or_pipeline=qna_model_large,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/326M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [10]:
print(results_tiny['f1'], results_tiny['HasAns_f1'], results_tiny['NoAns_f1'])
print(results_base['f1'], results_base['HasAns_f1'], results_base['NoAns_f1'])
print(results_large['f1'], results_large['HasAns_f1'], results_large['NoAns_f1'])

82.00862584399694 81.0093787318425 82.90258449304174
82.30597003199605 80.08353208998281 84.29423459244533
86.17319987933918 82.49568774446733 89.46322067594433


## BLEU

Medir o desempenho de modelos de tradução é um processo complexo, pois envolve a avaliação de aspectos subjetivos como compreensão, fidelidade e fluência da tradução dado que não existe uma única tradução possível para um texto. A métrica BLEU é uma forma automatizada de comparar a tradução gerada por um modelo de linguagem com uma ou, preferencialmente, múltiplas traduções de referência feitas por profissionais. O valor BLEU exprime o quão próximo a tradução gerada pelo modelo está das traduções de referência. Dessa forma, quanto maior o seu valor, melhor a tradução. É importante destacar que atingir o valor máximo (100) de pontuação é extremamente improvável, pois iria requerer que todas as traduções fossem idênticas a uma referência. Além disso, ao utilizar poucas referências de tradução para comparação o resultado tende a ter valores menores

In [11]:
en_pt_translation_dataset = load_dataset('opus_books', 'en-pt', split='train')

# Seleciona um subconjunto aleatório do dataset
n = len(en_pt_translation_dataset)
subset = random.choices(range(n), k=500)
en_pt_translation_dataset = en_pt_translation_dataset.select(subset)

english_texts = [translation_pair['en']
                 for translation_pair in en_pt_translation_dataset['translation']]

portuguese_texts = [translation_pair['pt']
                 for translation_pair in en_pt_translation_dataset['translation']]

en_pt_translation_dataset = en_pt_translation_dataset.add_column('english', english_texts)
en_pt_translation_dataset = en_pt_translation_dataset.add_column('portuguese', portuguese_texts)

README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/191k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1404 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
translation_model_opus = 'Helsinki-NLP/opus-mt-tc-big-en-pt'

sacrebleu_eval = evaluate.evaluator('translation')

results_opus = sacrebleu_eval.compute(model_or_pipeline=translation_model_opus,
             data=en_pt_translation_dataset,
             input_column='english',
             label_column='portuguese',
             metric='sacrebleu')

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/465M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/825k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [13]:
print(results_opus['score'])

39.561422863376414


# Questões

## 1) Qual a relação entre o número de parâmetros de um modelo e seu valor de perplexidade? O que isso significa em termos da capacidade de modelagem de linguagem do modelo?

 Conforme o número de parâmetros aumenta, o modelo consegue representar melhor os padrões linguísticos complexos e as relações contextuais nos dados. Isso aumenta sua capacidade de prever o próximo token com maior confiança, resultando em respostas mais coerentes e naturais, e uma redução na perplexidade. Em termos práticos, a menor perplexidade é um indicativo de que o modelo está mais ajustado para compreender e gerar respostas relevantes e alinhadas ao contexto, o que melhora sua performance em tarefas de linguagem natural.